# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use sports data **from year 2018** for your analysis but use populations data **from year 2016** -- this is important!

## Notes

1. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [129]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df.year ==2018]
    nhl_df['team'] = nhl_df['team'].str.replace(r"(\*)","")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    Mapper = {
        "Tampa Bay Lightning" : 'Tampa Bay Area',
        "Boston Bruins" : 'Boston',
        'Toronto Maple Leafs' : 'Toronto',
        'Florida Panthers' : 'Miami–Fort Lauderdale',
        'Detroit Red Wings' : 'Detroit',
        'Montreal Canadiens' : 'Montreal',
        'Ottawa Senators' : 'Ottawa',
        'Buffalo Sabres' : 'Buffalo',
        'Washington Capitals' : 'Washington, D.C.',
        'Pittsburgh Penguins' : 'Pittsburgh',
        'Philadelphia Flyers' : 'Philadelphia',
        'Columbus Blue Jackets' : 'Columbus',
        'New Jersey Devils' : 'New York City',
        'Carolina Hurricanes' : 'Raleigh',
        'New York Islanders' : 'New York City',
        'New York Rangers' : 'New York City',
        'Nashville Predators' :'Nashville',
        'Winnipeg Jets' : 'Winnipeg',
        'Minnesota Wild' : 'Minneapolis–Saint Paul',
        'Colorado Avalanche' : 'Denver',
        'St. Louis Blues' : 'St. Louis',
        'Dallas Stars' : 'Dallas–Fort Worth',
        'Chicago Blackhawks' : 'Chicago',
        'Vegas Golden Knights' : 'Las Vegas',
        'Anaheim Ducks' : 'Los Angeles',
        'San Jose Sharks' : 'San Francisco Bay Area',
        'Los Angeles Kings' : 'Los Angeles',
        'Calgary Flames' : 'Calgary',
        'Edmonton Oilers' : 'Edmonton',
        'Vancouver Canucks' : 'Vancouver',
        'Arizona Coyotes' : 'Phoenix'
    }
    
    nhl_df['Metropolitan Area'] = nhl_df.team.map(Mapper)
    nhl_df = nhl_df.set_index('Metropolitan Area')

    nhl_cities = pd.merge(nhl_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    nhl_cities['W'] = nhl_cities.W.astype(float)
    nhl_cities['L'] = nhl_cities.L.astype(float)
    nhl_cities['OL'] = nhl_cities.OL.astype(float)
    nhl_cities['Population (2016 est.)[8]'] = nhl_cities['Population (2016 est.)[8]'].astype(float)
    nhl_final = nhl_cities[['Metropolitan area', 'W', 'L', 'OL', 'Population (2016 est.)[8]']]
    nhl_final['ratio'] = nhl_final.W / (nhl_final.W + nhl_final.L)
    nhl_final = nhl_final.groupby(['Metropolitan area']).mean()

    population_by_region = nhl_final['Population (2016 est.)[8]']  # pass in metropolitan area population from cities
    win_loss_by_region = nhl_final.ratio  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [163]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df.year ==2018]
    nba_df['team'] = nba_df['team'].str.replace(r"[\*Â(\d)]","")
    nba_df.team = nba_df.team.str.strip()

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    Mapper = {
        'Boston Celtics' : 'Boston',
        'Toronto Raptors' : 'Toronto',
        'Miami Heat' : 'Miami–Fort Lauderdale',
        'Cleveland Cavaliers' : 'Cleveland',
        'Indiana Pacers' : 'Indianapolis',
        'Milwaukee Bucks' : 'Milwaukee',
        'Detroit Pistons' : 'Detroit',
        'Charlotte Hornets' : 'Charlotte',
        'Orlando Magic' : 'Orlando',
        'Atlanta Hawks': 'Atlanta',
        'Houston Rockets' : 'Houston',
        'Portland Trail Blazers' : 'Portland',
        'Oklahoma City Thunder' : 'Oklahoma City',
        'Utah Jazz' : 'Salt Lake City',
        'Washington Wizards' : 'Washington, D.C.',
        'New Orleans Pelicans' : 'New Orleans', 
        'San Antonio Spurs' : 'San Antonio',
        'Philadelphia ers' : 'Philadelphia',
        'New York Knicks' : 'New York City',
        'Brooklyn Nets' : 'New York City',
        'Minnesota Timberwolves': 'Minneapolis–Saint Paul',
        'Denver Nuggets' : 'Denver',
        'Dallas Mavericks' : 'Dallas–Fort Worth',
        'Chicago Bulls' : 'Chicago',
        'Los Angeles Lakers' : 'Los Angeles',
        'Golden State Warriors' : 'San Francisco Bay Area',
        'Los Angeles Clippers' : 'Los Angeles',
        'Phoenix Suns' : 'Phoenix',
        'Memphis Grizzlies' : 'Memphis', 
        'Sacramento Kings' : 'Sacramento'
    }
    
    nba_df['Metropolitan Area'] = nba_df.team.map(Mapper)
    nba_df = nba_df.set_index('Metropolitan Area')

    nba_cities = pd.merge(nba_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    nba_cities['W'] = nba_cities.W.astype(float)
    nba_cities['L'] = nba_cities.L.astype(float)
    nba_cities['Population'] = nba_cities['Population (2016 est.)[8]'].astype(float)
    nba_final = nba_cities[['Metropolitan area', 'W', 'L', 'Population']]
    nba_final['ratio'] = nba_final.W / (nba_final.W + nba_final.L)
    nba_final = nba_final.groupby(['Metropolitan area']).mean()

    
    population_by_region = nba_final.Population # pass in metropolitan area population from cities
    win_loss_by_region = nba_final.ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return  stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 

    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df.year ==2018]
    mlb_df.team = mlb_df.team.str.strip()

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    Mapper = {
        'Tampa Bay Rays' : 'Tampa Bay Area',
        'Boston Red Sox' : 'Boston',
        'Toronto Blue Jays' : 'Toronto',
        'Baltimore Orioles' : 'Baltimore',
        'Miami Marlins' : 'Miami–Fort Lauderdale',
        'Cleveland Indians' : 'Cleveland',
        'Milwaukee Brewers' : 'Milwaukee',
        'Detroit Tigers' : 'Detroit',
        'Atlanta Braves': 'Atlanta',
        'Washington Nationals' : 'Washington, D.C.',
        'Philadelphia Phillies' : 'Philadelphia',
        'New York Yankees' : 'New York City',
        'New York Mets' : 'New York City',
        'Minnesota Twins': 'Minneapolis–Saint Paul',
        'Colorado Rockies' : 'Denver',
        'Texas Rangers' : 'Dallas–Fort Worth',
        'Chicago White Sox' : 'Chicago',
        'Chicago Cubs' : 'Chicago',
        'Los Angeles Dodgers' : 'Los Angeles',
        'Oakland Athletics' : 'San Francisco Bay Area',
        'San Francisco Giants' : 'San Francisco Bay Area',
        'San Diego Padres' : 'San Diego',
        'Los Angeles Angels' : 'Los Angeles',
        'Arizona Diamondbacks' : 'Phoenix',
        'Kansas City Royals' : 'Kansas City',
        'Houston Astros' : 'Houston',
        'Seattle Mariners' : 'Seattle',
        'Pittsburgh Pirates' : 'Pittsburgh',
        'Cincinnati Reds' : 'Cincinnati',
        'St. Louis Cardinals' : 'St. Louis'
    }
    
    mlb_df['Metropolitan Area'] = mlb_df.team.map(Mapper)
    mlb_df = mlb_df.set_index('Metropolitan Area')

    mlb_cities = pd.merge(mlb_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    mlb_cities['W'] = mlb_cities.W.astype(float)
    mlb_cities['L'] = mlb_cities.L.astype(float)
    mlb_cities['Population'] = mlb_cities['Population (2016 est.)[8]'].astype(float)
    mlb_final = mlb_cities[['Metropolitan area', 'W', 'L', 'Population']]
    mlb_final['ratio'] = mlb_final.W / (mlb_final.W + mlb_final.L)
    mlb_final = mlb_final.groupby(['Metropolitan area']).mean()

    
    population_by_region = mlb_final.Population # pass in metropolitan area population from cities
    win_loss_by_region = mlb_final.ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [182]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    
    df=pd.read_csv("assets/nfl.csv")
    df = df[df.year ==2018]
    df['team'] = df['team'].str.replace(r"[\*+]","")
    df.team = df.team.str.strip()
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    Mapper = {
        'Tampa Bay Buccaneers' : 'Tampa Bay Area',
        'New England Patriots' : 'Boston',
        'Tennessee Titans' : 'Nashville',
        'Baltimore Ravens' : 'Baltimore',
        'Miami Dolphins' : 'Miami–Fort Lauderdale',
        'Cleveland Browns' : 'Cleveland',
        'Indianapolis Colts' : 'Indianapolis',
        'Green Bay Packers' : 'Green Bay',
        'Detroit Lions' : 'Detroit',
        'Carolina Panthers' : 'Charlotte',
        'Atlanta Falcons' : 'Atlanta',
        'Houston Texans' : 'Houston',
        'Washington Redskins' : 'Washington, D.C.',
        'New Orleans Saints' : 'New Orleans', 
        'Philadelphia Eagles' : 'Philadelphia',
        'New York Jets' : 'New York City',
        'New York Giants' : 'New York City',
        'Minnesota Vikings' : 'Minneapolis–Saint Paul',
        'Denver Broncos' : 'Denver',
        'Dallas Cowboys' : 'Dallas–Fort Worth',
        'Chicago Bears' : 'Chicago',
        'Jacksonville Jaguars' : 'Jacksonville',
        'Los Angeles Rams' : 'Los Angeles',
        'Oakland Raiders' : 'San Francisco Bay Area',
        'San Francisco 49ers' : 'San Francisco Bay Area',
        'Buffalo Bills' : 'Buffalo',
        'Los Angeles Chargers' : 'Los Angeles',
        'Arizona Cardinals' : 'Phoenix',
        'Kansas City Chiefs' : 'Kansas City',
        'Houston Texans' : 'Houston',
        'Seattle Seahawks' : 'Seattle',
        'Pittsburgh Steelers' : 'Pittsburgh',
        'Cincinnati Bengals' : 'Cincinnati'
    }
    
    df['Metropolitan Area'] = df.team.map(Mapper)
    df = df.set_index('Metropolitan Area')

    df_cities = pd.merge(df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    df_cities['W'] = df_cities.W.astype(float)
    df_cities['L'] = df_cities.L.astype(float)
    df_cities['Population'] = df_cities['Population (2016 est.)[8]'].astype(float)
    df_final = df_cities[['Metropolitan area', 'W', 'L', 'Population']]
    df_final['ratio'] = df_final.W / (df_final.W + df_final.L)
    df_final = df_final.groupby(['Metropolitan area']).mean()
 
    population_by_region = df_final.Population # pass in metropolitan area population from cities
    win_loss_by_region = df_final.ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [135]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_rel

import re

def sports_team_performance():
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df.year ==2018]
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df.year ==2018]
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df.year ==2018]
    df=pd.read_csv("assets/nfl.csv")
    df = df[df.year ==2018]
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    Mapper = {
        "Tampa Bay Lightning" : 'Tampa Bay Area',
         "Boston Bruins" : 'Boston',
         'Toronto Maple Leafs' : 'Toronto',
        'Florida Panthers' : 'Miami–Fort Lauderdale',
         'Detroit Red Wings' : 'Detroit',
         'Montreal Canadiens' : 'Montreal',
        'Ottawa Senators' : 'Ottawa',
        'Buffalo Sabres' : 'Buffalo',
        'Washington Capitals' : 'Washington, D.C.',
        'Pittsburgh Penguins' : 'Pittsburgh',
        'Philadelphia Flyers' : 'Philadelphia',
        'Columbus Blue Jackets' : 'Columbus',
        'New Jersey Devils' : 'New York City',
        'Carolina Hurricanes' : 'Raleigh',
        'New York Islanders' : 'New York City',
        'New York Rangers' : 'New York City',
        'Nashville Predators' :'Nashville',
        'Winnipeg Jets' : 'Winnipeg',
        'Minnesota Wild' : 'Minneapolis–Saint Paul',
        'Colorado Avalanche' : 'Denver',
        'St. Louis Blues' : 'St. Louis',
        'Dallas Stars' : 'Dallas–Fort Worth',
        'Chicago Blackhawks' : 'Chicago',
        'Vegas Golden Knights' : 'Las Vegas',
        'Anaheim Ducks' : 'Los Angeles',
        'San Jose Sharks' : 'San Francisco Bay Area',
        'Los Angeles Kings' : 'Los Angeles',
        'Calgary Flames' : 'Calgary',
        'Edmonton Oilers' : 'Edmonton',
        'Vancouver Canucks' : 'Vancouver',
        'Arizona Coyotes' : 'Phoenix',
        'Boston Celtics' : 'Boston',
        'Toronto Raptors' : 'Toronto',
        'Miami Heat' : 'Miami–Fort Lauderdale',
        'Cleveland Cavaliers' : 'Cleveland',
        'Indiana Pacers' : 'Indianapolis',
        'Milwaukee Bucks' : 'Milwaukee',
        'Detroit Pistons' : 'Detroit',
        'Charlotte Hornets' : 'Charlotte',
        'Orlando Magic' : 'Orlando',
        'Atlanta Hawks': 'Atlanta',
        'Houston Rockets' : 'Houston',
        'Portland Trail Blazers' : 'Portland',
        'Oklahoma City Thunder' : 'Oklahoma City',
        'Utah Jazz' : 'Salt Lake City',
        'Washington Wizards' : 'Washington, D.C.',
        'New Orleans Pelicans' : 'New Orleans', 
        'San Antonio Spurs' : 'San Antonio',
        'Philadelphia ers' : 'Philadelphia',
        'New York Knicks' : 'New York City',
        'Brooklyn Nets' : 'New York City',
        'Minnesota Timberwolves': 'Minneapolis–Saint Paul',
        'Denver Nuggets' : 'Denver',
        'Dallas Mavericks' : 'Dallas–Fort Worth',
        'Chicago Bulls' : 'Chicago',
        'Los Angeles Lakers' : 'Los Angeles',
        'Golden State Warriors' : 'San Francisco Bay Area',
        'Los Angeles Clippers' : 'Los Angeles',
        'Phoenix Suns' : 'Phoenix',
        'Memphis Grizzlies' : 'Memphis', 
        'Sacramento Kings' : 'Sacramento',
        'Tampa Bay Rays' : 'Tampa Bay Area',
        'Boston Red Sox' : 'Boston',
        'Toronto Blue Jays' : 'Toronto',
        'Baltimore Orioles' : 'Baltimore',
        'Miami Marlins' : 'Miami–Fort Lauderdale',
        'Cleveland Indians' : 'Cleveland',
        'Milwaukee Brewers' : 'Milwaukee',
        'Detroit Tigers' : 'Detroit',
        'Atlanta Braves': 'Atlanta',
        'Washington Nationals' : 'Washington, D.C.',
        'Philadelphia Phillies' : 'Philadelphia',
        'New York Yankees' : 'New York City',
        'New York Mets' : 'New York City',
        'Minnesota Twins': 'Minneapolis–Saint Paul',
        'Colorado Rockies' : 'Denver',
        'Texas Rangers' : 'Dallas–Fort Worth',
        'Chicago White Sox' : 'Chicago',
        'Chicago Cubs' : 'Chicago',
        'Los Angeles Dodgers' : 'Los Angeles',
        'Oakland Athletics' : 'San Francisco Bay Area',
        'San Francisco Giants' : 'San Francisco Bay Area',
        'San Diego Padres' : 'San Diego',
        'Los Angeles Angels' : 'Los Angeles',
        'Arizona Diamondbacks' : 'Phoenix',
        'Kansas City Royals' : 'Kansas City',
        'Houston Astros' : 'Houston',
        'Seattle Mariners' : 'Seattle',
        'Pittsburgh Pirates' : 'Pittsburgh',
        'Cincinnati Reds' : 'Cincinnati',
        'St. Louis Cardinals' : 'St. Louis',
        'Tampa Bay Buccaneers' : 'Tampa Bay Area',
        'New England Patriots' : 'Boston',
        'Tennessee Titans' : 'Nashville',
        'Baltimore Ravens' : 'Baltimore',
        'Miami Dolphins' : 'Miami–Fort Lauderdale',
        'Cleveland Browns' : 'Cleveland',
        'Indianapolis Colts' : 'Indianapolis',
        'Green Bay Packers' : 'Green Bay',
        'Detroit Lions' : 'Detroit',
        'Carolina Panthers' : 'Charlotte',
        'Atlanta Falcons' : 'Atlanta',
        'Houston Texans' : 'Houston',
        'Washington Redskins' : 'Washington, D.C.',
        'New Orleans Saints' : 'New Orleans', 
        'Philadelphia Eagles' : 'Philadelphia',
        'New York Jets' : 'New York City',
        'New York Giants' : 'New York City',
        'Minnesota Vikings' : 'Minneapolis–Saint Paul',
        'Denver Broncos' : 'Denver',
        'Dallas Cowboys' : 'Dallas–Fort Worth',
        'Chicago Bears' : 'Chicago',
        'Jacksonville Jaguars' : 'Jacksonville',
        'Los Angeles Rams' : 'Los Angeles',
        'Oakland Raiders' : 'San Francisco Bay Area',
        'San Francisco 49ers' : 'San Francisco Bay Area',
        'Buffalo Bills' : 'Buffalo',
        'Los Angeles Chargers' : 'Los Angeles',
        'Arizona Cardinals' : 'Phoenix',
        'Kansas City Chiefs' : 'Kansas City',
        'Houston Texans' : 'Houston',
        'Seattle Seahawks' : 'Seattle',
        'Pittsburgh Steelers' : 'Pittsburgh',
        'Cincinnati Bengals' : 'Cincinnati'
    }
    
    nhl_df['team'] = nhl_df['team'].str.replace(r"(\*)","")
    nhl_df['Metropolitan Area'] = nhl_df.team.map(Mapper)
    nhl_df = nhl_df.set_index('Metropolitan Area')
    nhl_cities = pd.merge(nhl_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    nhl_cities['W'] = nhl_cities.W.astype(float)
    nhl_cities['L'] = nhl_cities.L.astype(float)
    nhl_cities['OL'] = nhl_cities.OL.astype(float)
    nhl_cities['Population'] = nhl_cities['Population (2016 est.)[8]'].astype(float)
    nhl_final = nhl_cities[['Metropolitan area', 'W', 'L', 'OL', 'Population']]
    nhl_final['nhl_ratio'] = nhl_final.W / (nhl_final.W + nhl_final.L)
    nhl_final = nhl_final.groupby(['Metropolitan area']).mean()  
    
    nba_df['team'] = nba_df['team'].str.replace(r"[\*Â(\d)]","")
    nba_df.team = nba_df.team.str.strip()
    nba_df['Metropolitan Area'] = nba_df.team.map(Mapper)
    nba_df = nba_df.set_index('Metropolitan Area')
    nba_cities = pd.merge(nba_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    nba_cities['W'] = nba_cities.W.astype(float)
    nba_cities['L'] = nba_cities.L.astype(float)
    nba_cities['Population'] = nba_cities['Population (2016 est.)[8]'].astype(float)
    nba_final = nba_cities[['Metropolitan area', 'W', 'L', 'Population']]
    nba_final['nba_ratio'] = nba_final.W / (nba_final.W + nba_final.L)
    nba_final = nba_final.groupby(['Metropolitan area']).mean()
    
    mlb_df.team = mlb_df.team.str.strip()
    mlb_df['Metropolitan Area'] = mlb_df.team.map(Mapper)
    mlb_df = mlb_df.set_index('Metropolitan Area')
    mlb_cities = pd.merge(mlb_df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    mlb_cities['W'] = mlb_cities.W.astype(float)
    mlb_cities['L'] = mlb_cities.L.astype(float)
    mlb_cities['Population'] = mlb_cities['Population (2016 est.)[8]'].astype(float)
    mlb_final = mlb_cities[['Metropolitan area', 'W', 'L', 'Population']]
    mlb_final['mlb_ratio'] = mlb_final.W / (mlb_final.W + mlb_final.L)
    mlb_final = mlb_final.groupby(['Metropolitan area']).mean()
    
    df['team'] = df['team'].str.replace(r"[\*+]","")
    df.team = df.team.str.strip()
    df['Metropolitan Area'] = df.team.map(Mapper)
    df = df.set_index('Metropolitan Area')
    df_cities = pd.merge(df, cities, how='inner', left_index = True, right_on = 'Metropolitan area')
    df_cities['W'] = df_cities.W.astype(float)
    df_cities['L'] = df_cities.L.astype(float)
    df_cities['Population'] = df_cities['Population (2016 est.)[8]'].astype(float)
    df_final = df_cities[['Metropolitan area', 'W', 'L', 'Population']]
    df_final['nfl_ratio'] = df_final.W / (df_final.W + df_final.L)
    df_final = df_final.groupby(['Metropolitan area']).mean()
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    
    combined = pd.merge(nhl_final, nba_final, how='outer', left_on='Metropolitan area', right_on='Metropolitan area')
    combined = pd.merge(combined, mlb_final, how='outer', left_on='Metropolitan area', right_on='Metropolitan area')
    combined = pd.merge(combined, df_final, how='outer', left_on='Metropolitan area', right_on='Metropolitan area')
    combined = combined[['nfl_ratio', 'nba_ratio', 'nhl_ratio', 'mlb_ratio']]
    combined.columns = sports
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
            
    pval = {}
    for sport in sports:
        for column in combined.columns:
            if sport not in pval:
                pval[sport] = [ttest_rel(combined[column], combined[sport], nan_policy='omit')[1]]
            else:
                pval[sport].append(ttest_rel(combined[column], combined[sport], nan_policy='omit')[1])
                
    p_values = pd.DataFrame(pval, index=sports).astype(np.float64)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    return  p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
